# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keyv import geoapify_key

In [24]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
Cities_DF = pd.read_csv("../Starter_Code/starter_code/cities.csv")
#Cities_DFclean = Cities_DF[['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness','Wind Speed', 'Country', 'Date']]

# Display sample data
Cities_DF.head()

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
0,airai,TL,60.39,88,100,1.68,-8.9266,125.4092,1673721221
1,mataura,NZ,54.32,84,7,1.54,-46.1927,168.8643,1673721221
2,dikson,RU,-34.76,93,16,3.87,73.5069,80.5464,1673721221
3,luderitz,NaN,68.58,67,0,23.17,-26.6481,15.1594,1673721221
4,mount gambier,AU,59.23,65,100,13.18,-37.8333,140.7667,1673721131


In [25]:
#Cities_DF.columns

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

#configure gmaps
gmaps.configure(api_key=geoapify_key)

#get locations and humidity
city_locations = Cities_DF[['Latitude', 'Longitude']].astype(float)
humidity = Cities_DF['Humidity'].astype(float)

#format layout of map
layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'}

#add heatmap
fig = gmaps.figure(map_type='HYBRID', layout=layout)

heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

myFavoriteWeather = Cities_DF.loc[(Cities_DF['Temperature'] >= 70) & (Cities_DF['Temperature'] <= 80) & 
                                  (Cities_DF['Windspeed'] <= 20) & (Cities_DF['Humidity'] <= 30)]
myFavoriteWeather

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
32,araouane,ML,71.96,13,44,9.98,18.9048,-3.5265,1673721229
63,atar,MR,78.85,14,0,11.50,20.5169,-13.0499,1673721236
70,am timan,TD,79.14,17,14,3.44,11.0297,20.2827,1673721238
98,batouri,CM,78.94,22,5,2.13,4.4333,14.3667,1673721248
117,mpraeso,GH,78.08,27,1,3.44,6.5932,-0.7346,1673721255
263,sahuaripa,MX,79.75,16,90,2.95,29.0500,-109.2333,1673721296
326,adrar,MR,70.32,15,0,10.96,20.5022,-10.0711,1673721315
413,malakal,SS,78.96,21,0,5.79,9.5334,31.6605,1673721341
501,bossangoa,CF,78.78,21,11,4.21,6.4926,17.4552,1673721363
525,batangafo,CF,79.65,20,17,3.62,7.3008,18.2833,1673721376


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = myFavoriteWeather
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
0,araouane,ML,71.96,13,44,9.98,18.9048,-3.5265,1673721229
1,atar,MR,78.85,14,0,11.50,20.5169,-13.0499,1673721236
2,am timan,TD,79.14,17,14,3.44,11.0297,20.2827,1673721238
3,batouri,CM,78.94,22,5,2.13,4.4333,14.3667,1673721248
4,mpraeso,GH,78.08,27,1,3.44,6.5932,-0.7346,1673721255
5,sahuaripa,MX,79.75,16,90,2.95,29.0500,-109.2333,1673721296
6,adrar,MR,70.32,15,0,10.96,20.5022,-10.0711,1673721315
7,malakal,SS,78.96,21,0,5.79,9.5334,31.6605,1673721341
8,bossangoa,CF,78.78,21,11,4.21,6.4926,17.4552,1673721363
9,batangafo,CF,79.65,20,17,3.62,7.3008,18.2833,1673721376


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
hotel_df['Hotel Name'] = ''

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 50000,
    "rankby": "prominence",
    "types": "lodging",
    "keyword": "hotel",
    "api_keys": geoapify_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get variables type from df
    lat = row['Latitude']
    lng = row['Longitude']
    city = row['City']
    params["location"] = f'{lat},{lng}'
        
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {city} hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except:
        print(f"Missing field/result for {city}... skipping.")
        
    print("------------")

C:\Users\mcael\AppData\Local\Temp\ipykernel_23024\3448130591.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


Missing field/result for araouane... skipping.
------------
Missing field/result for atar... skipping.
------------
Missing field/result for am timan... skipping.
------------
Missing field/result for batouri... skipping.
------------
Missing field/result for mpraeso... skipping.
------------
Missing field/result for sahuaripa... skipping.
------------
Missing field/result for adrar... skipping.
------------
Missing field/result for malakal... skipping.
------------
Missing field/result for bossangoa... skipping.
------------
Missing field/result for batangafo... skipping.
------------
Missing field/result for jimenez... skipping.
------------


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display
hotel_df = hotel_df[hotel_df['Hotel Name'].notna()]
hotel_df = hotel_df.rename(columns={"Latitude": "Lat", "Longitude": "Lng"})
hotel_df

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Lat,Lng,Date,Hotel Name
0,araouane,ML,71.96,13,44,9.98,18.9048,-3.5265,1673721229,
1,atar,MR,78.85,14,0,11.50,20.5169,-13.0499,1673721236,
2,am timan,TD,79.14,17,14,3.44,11.0297,20.2827,1673721238,
3,batouri,CM,78.94,22,5,2.13,4.4333,14.3667,1673721248,
4,mpraeso,GH,78.08,27,1,3.44,6.5932,-0.7346,1673721255,
5,sahuaripa,MX,79.75,16,90,2.95,29.0500,-109.2333,1673721296,
6,adrar,MR,70.32,15,0,10.96,20.5022,-10.0711,1673721315,
7,malakal,SS,78.96,21,0,5.79,9.5334,31.6605,1673721341,
8,bossangoa,CF,78.78,21,11,4.21,6.4926,17.4552,1673721363,
9,batangafo,CF,79.65,20,17,3.62,7.3008,18.2833,1673721376,


In [31]:
# Using the template add the hotel marks to the heatmap
info_box_template = """

Name
{Hotel Name}

City
{City}
Country
{Country}

"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [32]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))